In [ ]:
import requests, zipfile, io, os
import geopandas as gpd
from owslib.wms import WebMapService

#top data service page with different options "Get the Data"
https://www.usgs.gov/centers/eros/news/annual-nlcd-lengthens-land-cover-record-40-years

#data service page
https://www.mrlc.gov/data-services-page

#link to data
https://dmsdata.cr.usgs.gov/geoserver/mrlc_Land-Cover-Native_conus_year_data/wms

In [ ]:
WMS_URL = "https://dmsdata.cr.usgs.gov/geoserver/mrlc_Land-Cover-Native_conus_year_data/wms"

wms = WebMapService(WMS_URL, version="1.3.0")

# See a few layer names (full list can be long)
layers = list(wms.contents)
print("Example layers:", layers[:10])

# Peek at one layer's metadata (replace with a layer you see above)
lyr = wms[ layers[0] ]
print("Title:", lyr.title)
print("CRS options:", lyr.crsOptions)
print("BoundingBox (native):", lyr.boundingBoxWGS84)

# What image formats does GetMap support?
print("GetMap formats:", wms.getOperationByName("GetMap").formatOptions)

In [ ]:
WMS_URL = "https://dmsdata.cr.usgs.gov/geoserver/mrlc_Land-Cover-Native_conus_year_data/wms"
LAYER    = "<PUT_A_LAYER_NAME_HERE>"   # e.g., something you found in step 1
CRS      = "EPSG:3857"                 # web mercator (common choice)
OUT_PNG  = "nlcd_example.png"

#get bbox extent
#host pinellas_county shapefile on github and download to memory
zip_url = "https://github.com/colarussoa/urban_flood_prediction/archive/refs/heads/main.zip"
response = requests.get(zip_url)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extractall("urban_flood_prediction_main")
shapefile_path = os.path.join(
    "urban_flood_prediction_main", 
    "urban_flood_prediction-main", 
    "Data", 
    "pinellas_county_shp", 
    "pinellas_county.shp"
)

aoi = gpd.read_file(shapefile_path).to_crs(CRS)
minx, miny, maxx, maxy = aoi.total_bounds
bbox_3857 = (minx, miny, maxx, maxy)

wms = WebMapService(WMS_URL, version="1.3.0")

img = wms.getmap(
    layers=[LAYER],
    styles=[""],             # usually empty style for NLCD products
    srs=CRS,                 # CRS for 1.3.0
    bbox=bbox_3857,          # (minx, miny, maxx, maxy) in EPSG:3857
    size=(2048, 2048),       # image pixels (adjust as needed)
    format="image/png",      # or 'image/geotiff' if supported by the service
    transparent=True
)

with open(OUT_PNG, "wb") as f:
    f.write(img.read())

print("Saved:", OUT_PNG)